In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint

### Datasets load

In [2]:
dataset_base_file_name = 'stack_overflow_datasets/survey_results_'
dataset_years = list(range(2011,2021))

In [3]:
# for the original dataset: encoding = "ISO-8859-1"
dfs = {}
for year in dataset_years:
    name = f'{dataset_base_file_name}{year}.csv'
    df = pd.read_csv(name, dtype=object)
    df['year'] = year
    dfs[year] = df

### Renaming column name to snake case format

In [4]:
import re
def camel_to_snake(name):
    # https://stackoverflow.com/a/1176023
    name = name.replace(' ', '_').replace("'", '').replace(':', '_').replace('?','')
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    name = re.sub('_+', r'_', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

for df in dfs.values():
    df.columns = map(lambda name: camel_to_snake(name), df.columns)

### Dataset columns comparison helpers

In [5]:
def print_year_dataset_cols_difference(base_year, years_against=[]):
    df_to_compare_with = dfs
    
    if years_against:
        df_to_compare_with = { year: dfs[year] for year in years_against }

    columns_to_compare = set(dfs[base_year].columns.tolist())

    for compared_year, compared_df in df_to_compare_with.items():
        if compared_year <= base_year: continue
        columns = compared_df.columns.tolist()
        print(f'{base_year} columns not present in {compared_year}')
        pprint(columns_to_compare - set(columns))
        print('\n')

In [6]:
def print_similar_col_name(columns: list, years=[]):
    df_to_compare_with = dfs
    if years:
        df_to_compare_with = { year: dfs[year] for year in years }

    for year, df in df_to_compare_with.items():
        print(year)
        similar = set()
        for name in columns:
            similar.update([col for col in df.columns.tolist() if name in col])
        pprint(similar)
        print('\n')

### Questions persisted over years

In [7]:
print_year_dataset_cols_difference(2011)

2011 columns not present in 2012
{'programming_languages',
 'programming_languages_oher',
 'recommendation_likely_acted_upon',
 'stackoverflow_sites_most_visited'}


2011 columns not present in 2013
{'programming_languages_oher',
 'project_type',
 'recommendation_likely_acted_upon',
 'stackoverflow_sites_most_visited'}


2011 columns not present in 2014
{'company_size',
 'job_satisfaction',
 'programming_languages_oher',
 'project_type',
 'recommendation_likely_acted_upon',
 'stackoverflow_sites_most_visited',
 'techn_related_purchases_last_year'}


2011 columns not present in 2015
{'annual_compensation',
 'company_size',
 'outside_expenditures_budget',
 'programming_languages',
 'programming_languages_oher',
 'project_type',
 'purchase_involvement',
 'purchase_involvement_type',
 'recommendation_likely_acted_upon',
 'stackoverflow_sites_most_visited',
 'tech_products_own',
 'tech_products_own_other',
 'techn_related_purchases_last_year',
 'usa_state'}


2011 columns not present in 201

### Column names similarity
Finding column names similarity based in some basic descriptors like:
* `data`, `age`, `year`, `gender`, `experience`, `time`, `want`, `lang`

In [8]:
print_similar_col_name(['language'], [])

2011
{'programming_languages', 'programming_languages_oher'}


2012
{'programming_language_other', 'programming_language'}


2013
{'programming_languages_other', 'programming_languages'}


2014
{'programming_languages_other', 'programming_languages'}


2015
{'programming_language',
 'programming_language_other',
 'want_work_language',
 'want_work_language_other'}


2016
set()


2017
{'want_work_language', 'have_worked_language'}


2018
{'language_desire_next_year', 'language_worked_with'}


2019
{'language_desire_next_year', 'language_worked_with'}


2020
{'language_desire_next_year', 'language_worked_with'}




### Dataset head exploration

In [9]:
dfs[2017].head(1)

,respondent,professional,program_hobby,country,university,employment_status,formal_education,major_undergrad,home_remote,company_size,...,gender,highest_education_parents,race,survey_long,questions_interesting,questions_confusing,interested_answers,salary,expected_salary,year
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN,2017


### Merging and transforming common columns

* 2018 occupation is expressed in both student and employment questions, so lets merge these two

In [10]:
dfs[2018].student.replace(['Yes, part-time', 'Yes, full-time', 'No'], ['student part-time', 'student full-time', 'Not student'], inplace=True)
dfs[2018]['occupation'] = dfs[2018].student + ';' + dfs[2018].employment

* 2017 remote status related questions `collaborate_remote, assess_job_remote, home_remote`

In [11]:
display(dfs[2017]['collaborate_remote'].unique())
display(dfs[2017]['assess_job_remote'].unique())
display(dfs[2017]['home_remote'].unique())

array([nan, 'Strongly disagree', 'Somewhat agree', 'Agree', 'Disagree',
       'Strongly agree'], dtype=object)

array(['Very important', nan, 'Somewhat important',
       'Not at all important', 'Important', 'Not very important'],
      dtype=object)

array([nan, 'More than half, but not all, the time',
       'Less than half the time, but at least one day each week', 'Never',
       "All or almost all the time (I'm full-time remote)",
       "It's complicated", 'A few days each month', 'About half the time'],
      dtype=object)

### Datasets columns renaming
After exploring common names here's the final mapping.

In [12]:
for df in dfs.values():
    columns_renamed={
        'salary': 'annual_compensation',
        'annueal_compensation': 'annual_compensation',
        
        'org_size': 'company_size',
        
        'years_code': 'years_experience', 
        'years_coding': 'years_experience', 
        'years_program': 'years_experience', 
        
        'main_branch': 'occupation', 
        'professional': 'occupation',
        
        'training_and_education': 'education',
        'education_types': 'education',
        'ed_level': 'education',
        
        'remote': 'remote_status',
        'work_remote': 'remote_status',
        'home_remote': 'remote_status',
        
        'job_sat': 'job_satisfaction',
        
        'op_sys': 'os',
        'operating_system': 'os',
        'desktop_os': 'os',
        
        'age_range': 'age',
        'company_size_range': 'company_size',
        'experience_range': 'years_experience',
        'salary_range': 'annual_compensation',
        'team_size_range': 'team_size',
        
        'language_worked_with': 'programming_languages',
        'have_worked_language': 'programming_languages',
        'tech_do': 'programming_languages',
        'programming_lalnguages': 'programming_languages',
        'programming_language': 'programming_languages',
        'programming_languages_oher': 'programming_languages_other',
        'programming_language_other': 'programming_languages_other',
        
        'future_language_or_tech': 'want_work_language',
        'tech_want': 'want_work_language',
        'language_desire_next_year': 'want_work_language',
        'new_tech_interests': 'want_work_language',
        
        'have_worked_database': 'database_worked_with',
        'want_work_database': 'database_desire_work',
        'database_desire_next_year': 'database_desire_work',
        
        'major_undergrad': 'undergrad_mayor', 
 
        'so_visit1st': 'stack_overflow_visit1st',
        'so_visit_freq': 'stack_overflow_visit_freq',
        'so_visit_to': 'stack_overflow_visit_to',
        'so_find_answer': 'stack_overflow_find_answer',
        'so_time_saved': 'stack_overflow_time_saved',
        'so_how_much_time': 'stack_overflow_how_much_time',
        'so_account': 'stack_overflow_account',
        'so_part_freq': 'stack_overflow_part_freq',
        'so_jobs': 'stack_overflow_jobs',
        'so_comm': 'stack_overflow_comm',
        'so_new_content': 'stack_overflow_new_content',
        'so_region': 'stack_overflow_region',
        'newso_sites': 'new_stack_overflow_sites'
    }
    df.rename(columns=columns_renamed, inplace=True)

### Questions over time
After exploring some common questions persisted over time, here's the final list of questions that will be used for analysis:

`P: present`  
`NP: not present`  
`AP: always present`

* `programming_languages`: AP
* `country`: AP
* `years_experience`: AP
* `occupation`: AP


* `age`: AP but 2017
* `os`: AP but 2017
* `job_satisfaction`: AP but 2014
* `company_size`: AP but 2014, 2015


* `want_work_language`: P since 2013
* `gender`: P since 2014
* `education`: P since 2015
* `database_desire_work`: P since 2017
* `database_worked_with`: P since 2017
* `undergrad_major`: P since 2018


* `remote_status`: P 2014-2017, 2019


* `industry`: NP since 2017

### Missing values per row per dataset
Feature column for tracking percetage of missing rows per year dataset.

In [13]:
for df in dfs.values():
    presence_values_percentage = df.apply(lambda x: x.count(), axis=1)/df.shape[1]*100
    df['original_missing_values_per_row_percentage'] = 100 - presence_values_percentage
    df['original_missing_values_per_row_percentage'] = df['original_missing_values_per_row_percentage'].round(2)

In [14]:
dfs[2011]['original_missing_values_per_row_percentage'].value_counts().sort_index()

0.00      13
4.55     243
9.09     760
13.64    839
18.18    314
22.73    303
27.27     62
31.82     23
36.36     21
40.91      9
45.45     11
50.00      8
54.55     22
59.09     18
72.73     15
77.27     68
86.36     11
90.91     73
Name: original_missing_values_per_row_percentage, dtype: int64

### Merging datasets common questions

In [15]:
features = [
    'year', 'original_missing_values_per_row_percentage',
    'country', 'programming_languages', 'programming_languages_other', 'years_experience', 'occupation',
    'age', 'os', 'job_satisfaction', 'company_size',
    'want_work_language', 'gender', 'education', 'database_desire_work', 'database_worked_with', 'undergrad_major',
    'remote_status',
    'industry'
]

In [16]:
concat_df = pd.concat([df for df in dfs.values()], ignore_index=True)[features]

In [17]:
concat_df.to_csv('survey_report_concat_common_questions.csv', index=False)